In [ ]:
#Objectif : créer une base qui regroupe les données sur la représentation politique des femmes
#Données récupérées sur le site de l'union interparlementaire : https://data.ipu.org/fr/women-ranking?month=11&year=2021

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import bs4
import pandas
from urllib import request
from scipy import stats
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model

# Création de la base de données Représentation Politique

## Données sur la proportion de femmes au parlement par pays (en fonction de la chambre)

In [40]:
df_percent_women = pd.read_csv('C:/Users/gadole/Documents/cours/Ensae/2A/python/projet/RP/chamber--current_women_percent.csv', sep=',', encoding='latin-1')
df_percent_women.head()

,Source: Inter-Parliamentary Union Open Data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Date: 2021-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Classement au 1er Novembre 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Classement,Pays,Chambre basse ou unique,NaN,NaN,NaN,Chambre haute,NaN,NaN,NaN
4,NaN,NaN,Elections,SiÃ¨ges,Femmes,% W,Elections,SiÃ¨ges,Femmes,% W


In [41]:
df_percent_women = df_percent_women.iloc[3: , : ]
df_percent_women.drop(df_percent_women.columns[[0]], axis=1, inplace=True)
df_percent_women.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
3,Pays,Chambre basse ou unique,NaN,NaN,NaN,Chambre haute,NaN,NaN,NaN
4,NaN,Elections,SiÃ¨ges,Femmes,% W,Elections,SiÃ¨ges,Femmes,% W
5,Rwanda,09.2018,80,49,61.3,09.2019,26,10,38.5
6,Cuba,03.2018,586,313,53.4,-,-,-,-
7,Nicaragua,11.2016,91,46,50.6,-,-,-,-


In [42]:
df_percent_women.columns = ["Country Name", "Elections (lower house)", "Seats", "Women", "Women proportion", "Elections (upper house)", "Seats", "Women", "Women proportion"]
df_percent_women = df_percent_women.iloc[2: , : ]
df_percent_women.reset_index(inplace=True, drop=True)
df_percent_women.head()

,Country Name,Elections (lower house),Seats,Women,Women proportion,Elections (upper house),Seats,Women,Women proportion
0,Rwanda,09.2018,80,49,61.3,09.2019,26,10,38.5
1,Cuba,03.2018,586,313,53.4,-,-,-,-
2,Nicaragua,11.2016,91,46,50.6,-,-,-,-
3,Emirats arabes unis,10.2019,40,20,50,-,-,-,-
4,Mexique,06.2021,500,250,50,07.2018,128,63,49.2


## Données sur l'existence de quota électoraux concernant les femmes

In [145]:
df_quota = pd.read_csv('C:/Users/gadole/Documents/cours/Ensae/2A/python/projet/RP/chamber--field_is_electoral_quota_women.csv', sep=',', encoding='latin-1')
df_quota.head()

,Source : base de donnÃ©es Parline sur les parlements nationaux (https://data.ipu.org).,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Droit d'auteur : Union interparlementaire. Con...,NaN,NaN,NaN,NaN,NaN,NaN
1,Champ sÃ©lectionnÃ© : Quota Ã©lectoral de femmes,NaN,NaN,NaN,NaN,NaN,NaN
2,AnnÃ©e de rÃ©fÃ©rence : 2021,NaN,NaN,NaN,NaN,NaN,NaN
3,Exported on: 2021-12-27,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
def nettoyage(df, d, l):
    #df est le data frame
    #d représente l'indice des colonnes à supprimer
    #l représente l'indice de lignes à partir duquel il convient de conserver les données
    df = df.drop(df.columns[d], axis=1) #On supprime les colonnes vides
    df = df.iloc[l: , : ] #On conserve les lignes comportant les noms des colonnes et les données
    df.columns = df.iloc[0] #On renomme les colonnes
    df.reset_index(inplace=True, drop=True) 
    df = df.iloc[pd.RangeIndex(len(df)).drop(0)] 
    df.reset_index(inplace=True, drop=True)
    return df

In [147]:
df_quota = nettoyage(df_quota, 6, 5)
df_quota.head()

5,Code ISO,Pays,Chambre,Quota Ã©lectoral de femmes,Structure,Type de chambre
0,AF,Afghanistan,Chambre du peuple,Oui,Bicameral,Chambre basse
1,ZA,Afrique du Sud,AssemblÃ©e nationale,Oui,Bicameral,Chambre basse
2,AL,Albanie,Parlement,Oui,Unicameral,NaN
3,DZ,AlgÃ©rie,AssemblÃ©e populaire nationale,Oui,Bicameral,Chambre basse
4,DE,Allemagne,Bundestag allemand,Oui,Bicameral,Chambre basse


In [148]:
df_quota.columns = ["Country Code", "Country", "House", "Quota", "Structure", "House type"]
df_quota.head()

,Country Code,Country,House,Quota,Structure,House type
0,AF,Afghanistan,Chambre du peuple,Oui,Bicameral,Chambre basse
1,ZA,Afrique du Sud,AssemblÃ©e nationale,Oui,Bicameral,Chambre basse
2,AL,Albanie,Parlement,Oui,Unicameral,NaN
3,DZ,AlgÃ©rie,AssemblÃ©e populaire nationale,Oui,Bicameral,Chambre basse
4,DE,Allemagne,Bundestag allemand,Oui,Bicameral,Chambre basse


## Existence d'un organe spécialisé sur la question de l'égalité des sexes

In [151]:
df_spe_body = pd.read_csv('C:/Users/gadole/Documents/cours/Ensae/2A/python/projet/RP/chamber--specialized_body__women.csv', sep=',', encoding='latin-1')
df_spe_body.head()

,Source : base de donnÃ©es Parline sur les parlements nationaux (https://data.ipu.org).,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Droit d'auteur : Union interparlementaire. Con...,NaN,NaN,NaN,NaN,NaN,NaN
1,Field name: Organes spÃ©cialisÃ©s - EgalitÃ© d...,NaN,NaN,NaN,NaN,NaN,NaN
2,AnnÃ©e de rÃ©fÃ©rence : 2021,NaN,NaN,NaN,NaN,NaN,NaN
3,Exported on: 2021-12-27,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
df_spe_body = nettoyage(df_spe_body, 6, 5)
df_spe_body.head()

5,Code ISO,Pays,Chambre,Organes spÃ©cialisÃ©s - EgalitÃ© des sexes,Structure,Type de chambre
0,AF,Afghanistan,Chambre du peuple,Oui,Bicameral,Chambre basse
1,ZA,Afrique du Sud,AssemblÃ©e nationale,Oui,Bicameral,Chambre basse
2,AL,Albanie,Parlement,Oui,Unicameral,NaN
3,DZ,AlgÃ©rie,AssemblÃ©e populaire nationale,Non,Bicameral,Chambre basse
4,DE,Allemagne,Bundestag allemand,Oui,Bicameral,Chambre basse
